# Online Retail: Data Exploration Notebook

## Source  
Dr Daqing Chen, Director: Public Analytics group. chend '@' lsbu.ac.uk, School of Engineering, London South Bank University, London SE1 0AA, UK.

## Goals and Questions  
* What items do customers buy most?  
* Are there distinct groups amongst the customer base based upon purchasing patterns?

## Packages/Modules Used

In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from sklearn.cluster import Birch
from dotenv import load_dotenv
load_dotenv()

True

## Data Processing

In [2]:
online_retail_data = pd.read_csv(f"{os.getenv('PROJ_REPOS')}\\data\\Online_Retail.csv", encoding='latin1')
online_retail_data.sample(n=10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
181967,552514,DOT,DOTCOM POSTAGE,1,5/9/2011 16:30,205.45,NaN,United Kingdom
137990,548178,85123A,WHITE HANGING HEART T-LIGHT HOLDER,12,3/29/2011 14:35,2.95,14367.0,United Kingdom
280465,561453,22671,FRENCH LAUNDRY SIGN BLUE METAL,12,7/27/2011 11:47,1.65,13183.0,United Kingdom
114889,546091,84978,HANGING HEART JAR T-LIGHT HOLDER,12,3/9/2011 11:53,1.25,14936.0,Channel Islands
68644,541871,21401,BLUE PUDDING SPOON,1,1/24/2011 9:41,0.83,NaN,United Kingdom
69799,541986,22560,TRADITIONAL MODELLING CLAY,24,1/24/2011 15:59,1.25,17001.0,United Kingdom
519089,580136,23582,VINTAGE DOILY JUMBO BAG RED,1,12/1/2011 19:42,2.08,17841.0,United Kingdom
328700,565801,23236,STORAGE TIN VINTAGE DOILY,6,9/7/2011 10:28,2.89,12685.0,France
287193,562093,85038,6 CHOCOLATE LOVE HEART T-LIGHTS,18,8/2/2011 12:59,1.85,13777.0,United Kingdom
58880,541265,16207B,PINK HEART RED HANDBAG,1,1/16/2011 16:23,2.95,17609.0,United Kingdom


In [3]:
customer_purchases = pd.read_csv(f"{os.getenv('PROJ_REPOS')}\\data\\Customer_Purchases_Clusters.csv", encoding='latin1')
customer_purchases

,CustomerID,Cluster,Australia,Austria,Bahrain,Belgium,Brazil,Canada,Channel_Islands,Cyprus,...,90214O,90214P,90214R,90214S,90214T,90214U,90214V,90214W,90214Y,90214Z
0,12346,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,12347,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12348,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,12349,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,12350,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4367,18280,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4368,18281,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4369,18282,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4370,18283,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
customer_purchases_norm = pd.read_csv(f"{os.getenv('PROJ_REPOS')}\\data\\Customer_Purchases_Clusters_Norm.csv", encoding='latin1')
customer_purchases_norm

,CustomerID,Cluster,Australia,Austria,Bahrain,Belgium,Brazil,Canada,Channel_Islands,Cyprus,...,90214O,90214P,90214R,90214S,90214T,90214U,90214V,90214W,90214Y,90214Z
0,12346,3,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12347,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12348,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12349,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12350,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4367,18280,3,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4368,18281,3,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4369,18282,3,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4370,18283,3,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
stock_codes = list(set(online_retail_data.StockCode.astype(str).values))
stock_codes.sort()
for i, stock_code in enumerate(stock_codes):
    stock_codes[i] = stock_code.replace(' ', '_')
stock_codes = stock_codes[:-8]

In [43]:
cluster0 = customer_purchases.query('Cluster == 0').set_index('CustomerID')
cluster0 = cluster0.loc[:, (cluster0 != 0).any(axis=0)]
cluster0['Cluster'] = 0
to_front_col = cluster0.pop('Cluster')
cluster0.insert(0, 'Cluster', to_front_col)
filter_col = [col for col in cluster0 if col in stock_codes]
cluster0[filter_col] = cluster0[filter_col].div(cluster0[filter_col].sum(axis=1), axis=0)
cluster0 = cluster0.dropna()
cluster0

,Cluster,Australia,Austria,Bahrain,Belgium,Brazil,Canada,Channel_Islands,Cyprus,Czech_Republic,...,90164A,90166,90177D,90192,90201A,90201B,90201C,90201D,90205A,90205C
CustomerID,,,,,,,,,,,,,,,,,,,,,
12347,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12348,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12349,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12350,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12352,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17444,0,0,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17508,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17828,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
filter_col = [col for col in cluster0 if col in stock_codes]
cluster0.groupby('Cluster').sum()[filter_col]

,10002,10125,10133,10135,11001,15034,15036,15039,15044A,15044B,...,90164A,90166,90177D,90192,90201A,90201B,90201C,90201D,90205A,90205C
Cluster,,,,,,,,,,,,,,,,,,,,,
0,0.071333,0.00819,0.062994,0.018775,0.040772,0.145166,0.159448,0.007622,0.002833,0.017291,...,0.001321,0.000039,0.000078,0.003791,0.000078,0.000078,0.000078,0.000078,0.000052,0.000052


In [56]:
cluster0.groupby('Cluster').sum()[filter_col]

Cluster
0    2.611571
dtype: float64

In [32]:
cluster1 = customer_purchases.query('Cluster == 1').set_index('CustomerID')
cluster1 = cluster1.loc[:, (cluster1 != 0).any(axis=0)]
filter_col = [col for col in cluster1 if col in stock_codes]
cluster1[filter_col] = cluster1[filter_col].div(cluster1[filter_col].sum(axis=1), axis=0)
cluster1 = cluster1.dropna()
cluster1

,Cluster,France,10002,10120,10125,10135,11001,15036,15039,15044C,...,85230E,85230G,85232D,90030B,90030C,90031,90099,90184B,90184C,90201B
CustomerID,,,,,,,,,,,,,,,,,,,,,
12413,1,1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12437,1,1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12441,1,1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12488,1,1,0.0,0.0,0.000000,0.019011,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12489,1,1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12734,1,1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12735,1,1,0.0,0.0,0.055096,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12736,1,1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
cluster2 = customer_purchases.query('Cluster == 2').set_index('CustomerID')
cluster2 = cluster2.loc[:, (cluster2 != 0).any(axis=0)]
filter_col = [col for col in cluster2 if col in stock_codes]
cluster2[filter_col] = cluster2[filter_col].div(cluster2[filter_col].sum(axis=1), axis=0)
cluster2 = cluster2.dropna()
cluster2

,Cluster,Germany,10002,10125,10135,11001,15034,15036,15039,15044A,...,90161C,90161D,90170,90173,90201A,90201B,90201C,90201D,90202D,90204
CustomerID,,,,,,,,,,,,,,,,,,,,,
12426,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12427,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12468,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12471,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12472,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13814,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13815,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13816,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
cluster3 = customer_purchases.query('Cluster == 3').set_index('CustomerID')
cluster3 = cluster3.loc[:, (cluster3 != 0).any(axis=0)]
filter_col = [col for col in cluster3 if col in stock_codes]
cluster3[filter_col] = cluster3[filter_col].div(cluster3[filter_col].sum(axis=1), axis=0)
cluster3 = cluster3.dropna()
cluster3

,Cluster,United_Kingdom,10002,10080,10120,10123C,10124A,10124G,10125,10133,...,90214O,90214P,90214R,90214S,90214T,90214U,90214V,90214W,90214Y,90214Z
CustomerID,,,,,,,,,,,,,,,,,,,,,
12747,3,1,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12748,3,1,0.000042,0.0,0.00025,0.0,0.0,0.0,0.0,0.001166,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12749,3,1,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12820,3,1,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12821,3,1,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280,3,1,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18281,3,1,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18282,3,1,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
